<a href="https://colab.research.google.com/github/hahachang/colab/blob/main/Pasre_TWSE_FinancialStatment(web).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install beautifulsoup4
from bs4 import BeautifulSoup
import requests
import pandas as pd

#### new table format ####
from google.colab import data_table
from vega_datasets import data

data_table.enable_dataframe_formatter()

 # 設定參數

In [2]:
#@title 基本參數
stock =2330 #@param {type:"raw"}
year =2021 #@param {type:"raw"}
quarter = 3 #@param [1, 2, 3, 4] {type:"raw"}

stock   = str(stock)
year    = str(year)
quarter = str(quarter)

In [3]:
url = "https://mops.twse.com.tw/server-java/t164sb01?step="+ quarter + "&SYEAR=" + year +"&file_name=tifrs-fr1-m1-ci-cr-"+stock+"-"+year+"Q"+quarter+".html"
#url = "https://mops.twse.com.tw/server-java/t164sb01?step=3&SYEAR=2021&file_name=tifrs-fr1-m1-ci-cr-2330-2021Q3.html"

# 抓資料

In [4]:
res = requests.get(url)
res.encoding = 'big5'
soup = BeautifulSoup(res.text, 'html.parser')

# 測試

In [ ]:
table = soup.find_all('table')[0]

In [ ]:
tablename = [th.text.replace('\n', '') for th in table.find_all('tr')[0].find_all('th')]
tablename

['資產負債表Balance Sheet']

In [ ]:
columns = [th.text.replace('\n', '') for th in table.find_all('tr')[1].find_all('th')]
columns

['代號Code',
 '會計項目Accounting Title',
 '2021年9月30日2021/9/30',
 '2020年12月31日2020/12/31',
 '2020年9月30日2020/9/30']

In [ ]:
trs = table.find_all('tr')[2:]
rows = list()
for tr in trs:
    rows.append([td.text.replace('\n', '').replace('\xa0', '') for td in tr.find_all('td')])
#rows[:5]

df = pd.DataFrame(data=rows, columns=columns)
df

,代號Code,會計項目Accounting Title,2021年9月30日2021/9/30,2020年12月31日2020/12/31,2020年9月30日2020/9/30
0,,資產Assets,,,
1,,流動資產 Current assets,,,
2,1100,現金及約當現金 Cash and cash equivalents,"853,811,966","660,170,647","604,195,113"
3,1110,透過損益按公允價值衡量之金融資產－流動 Current financial asset...,"33,956","2,259,412","2,526,481"
4,1120,透過其他綜合損益按公允價值衡量之金融資產－流動 Current financial a...,"121,935,878","122,448,453","128,608,752"
...,...,...,...,...,...
83,36XX,非控制權益 Non-controlling interests,"2,300,248","964,743","905,346"
84,3XXX,權益總額 Total equity,"2,078,320,310","1,850,621,999","1,788,266,375"
85,3X2X,負債及權益總計 Total liabilities and equity,"3,332,311,884","2,760,711,405","2,635,572,214"
86,3998,預收股款（權益項下）之約當發行股數 Equivalent issue shares of ...,0,0,0


# 資產負債表

In [5]:
table_i = 0
table = soup.find_all('table')[table_i]

tablename = [th.text.replace('\n', '') for th in table.find_all('tr')[0].find_all('th')]
tablename

columns = [th.text.replace('\n', '') for th in table.find_all('tr')[1].find_all('th')]
columns

trs = table.find_all('tr')[2:]
rows = list()
for tr in trs:
    rows.append([td.text.replace('\n', '').replace('\xa0', '') for td in tr.find_all('td')])
#rows[:5]

df = pd.DataFrame(data=rows, columns=columns)
df.head()


,代號Code,會計項目Accounting Title,2021年9月30日2021/9/30,2020年12月31日2020/12/31,2020年9月30日2020/9/30
0,,資產Assets,,,
1,,流動資產 Current assets,,,
2,1100,現金及約當現金 Cash and cash equivalents,"853,811,966","660,170,647","604,195,113"
3,1110,透過損益按公允價值衡量之金融資產－流動 Current financial asset...,"33,956","2,259,412","2,526,481"
4,1120,透過其他綜合損益按公允價值衡量之金融資產－流動 Current financial a...,"121,935,878","122,448,453","128,608,752"


In [ ]:
tablename

['資產負債表Balance Sheet']

In [ ]:
df.insert(loc=0, column='sheet'    , value=tablename[0])
df.insert(loc=0, column='stockcode', value=stockcode)

In [ ]:
dfs = pd.melt(df,id_vars=['sheet','stockcode','代號Code','會計項目Accounting Title'])
dfs

,sheet,stockcode,代號Code,會計項目Accounting Title,variable,value
0,資產負債表Balance Sheet,2330,,資產Assets,2021年3月31日2021/3/31,
1,資產負債表Balance Sheet,2330,,流動資產 Current assets,2021年3月31日2021/3/31,
2,資產負債表Balance Sheet,2330,1100,現金及約當現金 Cash and cash equivalents,2021年3月31日2021/3/31,"664,727,857"
3,資產負債表Balance Sheet,2330,1110,透過損益按公允價值衡量之金融資產－流動 Current financial asset...,2021年3月31日2021/3/31,"26,897"
4,資產負債表Balance Sheet,2330,1120,透過其他綜合損益按公允價值衡量之金融資產－流動 Current financial a...,2021年3月31日2021/3/31,"125,208,307"
...,...,...,...,...,...,...
262,資產負債表Balance Sheet,2330,36XX,非控制權益 Non-controlling interests,2020年3月31日2020/3/31,"762,531"
263,資產負債表Balance Sheet,2330,3XXX,權益總額 Total equity,2020年3月31日2020/3/31,"1,677,028,531"
264,資產負債表Balance Sheet,2330,3X2X,負債及權益總計 Total liabilities and equity,2020年3月31日2020/3/31,"2,343,295,696"
265,資產負債表Balance Sheet,2330,3998,預收股款（權益項下）之約當發行股數 Equivalent issue shares of ...,2020年3月31日2020/3/31,0


# 現金流量表

In [13]:
table_i = 2
table = soup.find_all('table')[table_i]

tablename = [th.text.replace('\n', '') for th in table.find_all('tr')[0].find_all('th')]
columns = [th.text.replace('\n', '') for th in table.find_all('tr')[1].find_all('th')]

trs = table.find_all('tr')[2:]
rows = list()
for tr in trs:
    rows.append([td.text.replace('\n', '').replace('\xa0', '') for td in tr.find_all('td')])

df = pd.DataFrame(data=rows, columns=columns)

title     = soup.select('body span.zh')
stockcode = title[0].text.split(" ")[0]

df.insert(loc=0, column='sheet'    , value=tablename[0])
df.insert(loc=0, column='stockcode', value=stockcode)

df = pd.melt(df,id_vars=['stockcode','sheet','代號Code','會計項目Accounting Title'])
df

,stockcode,sheet,代號Code,會計項目Accounting Title,variable,value
0,2330,現金流量表Statements of Cash Flows,,營業活動之現金流量－間接法Cash flows from (used in) operati...,2021年1月1日至9月30日2021/1/1To9/30,
1,2330,現金流量表Statements of Cash Flows,A00010,繼續營業單位稅前淨利（淨損） Profit (loss) from continuing ...,2021年1月1日至9月30日2021/1/1To9/30,"478,307,103"
2,2330,現金流量表Statements of Cash Flows,A10000,本期稅前淨利（淨損） Profit (loss) before tax,2021年1月1日至9月30日2021/1/1To9/30,"478,307,103"
3,2330,現金流量表Statements of Cash Flows,,調整項目 Adjustments,2021年1月1日至9月30日2021/1/1To9/30,
4,2330,現金流量表Statements of Cash Flows,,收益費損項目 Adjustments to reconcile profit (loss),2021年1月1日至9月30日2021/1/1To9/30,
...,...,...,...,...,...,...
161,2330,現金流量表Statements of Cash Flows,DDDD,匯率變動對現金及約當現金之影響Effect of exchange rate changes...,2020年1月1日至9月30日2020/1/1To9/30,"(12,505,458)"
162,2330,現金流量表Statements of Cash Flows,EEEE,本期現金及約當現金增加（減少）數Net increase (decrease) in cas...,2020年1月1日至9月30日2020/1/1To9/30,"148,795,777"
163,2330,現金流量表Statements of Cash Flows,E00100,期初現金及約當現金餘額Cash and cash equivalents at beginn...,2020年1月1日至9月30日2020/1/1To9/30,"455,399,336"
164,2330,現金流量表Statements of Cash Flows,E00200,期末現金及約當現金餘額Cash and cash equivalents at end of...,2020年1月1日至9月30日2020/1/1To9/30,"604,195,113"


In [14]:
df['variable'] = df['variable'].str.extract('(\d\d\d\d)年',expand=False)+"/"+df['variable'].str.extract('至(\d*)月',expand=False)
df['sheet']=df['sheet'].str.replace("Statements of Cash Flows","")
df.value=df.value.str.replace(',','')
df.value=df.value.str.replace('(',"-")
df.value=df.value.str.replace(')','')
df

,stockcode,sheet,代號Code,會計項目Accounting Title,variable,value
0,2330,現金流量表,,營業活動之現金流量－間接法Cash flows from (used in) operati...,2021/9,
1,2330,現金流量表,A00010,繼續營業單位稅前淨利（淨損） Profit (loss) from continuing ...,2021/9,478307103
2,2330,現金流量表,A10000,本期稅前淨利（淨損） Profit (loss) before tax,2021/9,478307103
3,2330,現金流量表,,調整項目 Adjustments,2021/9,
4,2330,現金流量表,,收益費損項目 Adjustments to reconcile profit (loss),2021/9,
...,...,...,...,...,...,...
161,2330,現金流量表,DDDD,匯率變動對現金及約當現金之影響Effect of exchange rate changes...,2020/9,-12505458
162,2330,現金流量表,EEEE,本期現金及約當現金增加（減少）數Net increase (decrease) in cas...,2020/9,148795777
163,2330,現金流量表,E00100,期初現金及約當現金餘額Cash and cash equivalents at beginn...,2020/9,455399336
164,2330,現金流量表,E00200,期末現金及約當現金餘額Cash and cash equivalents at end of...,2020/9,604195113


In [15]:
df.query('代號Code == "AAAA" | 代號Code == "BBBB" | 代號Code == "CCCC" | 代號Code == "DDDD" | 代號Code == "EEEE"')


,stockcode,sheet,代號Code,會計項目Accounting Title,variable,value
44,2330,現金流量表,AAAA,營業活動之淨現金流入（流出）Net cash flows from (used in) op...,2021/9,733961405
61,2330,現金流量表,BBBB,投資活動之淨現金流入（流出） Net cash flows from (used in) ...,2021/9,-591022571
77,2330,現金流量表,CCCC,籌資活動之淨現金流入（流出） Net cash flows from (used in) ...,2021/9,54324420
78,2330,現金流量表,DDDD,匯率變動對現金及約當現金之影響Effect of exchange rate changes...,2021/9,-3621935
79,2330,現金流量表,EEEE,本期現金及約當現金增加（減少）數Net increase (decrease) in cas...,2021/9,193641319
127,2330,現金流量表,AAAA,營業活動之淨現金流入（流出）Net cash flows from (used in) op...,2020/9,563535628
144,2330,現金流量表,BBBB,投資活動之淨現金流入（流出） Net cash flows from (used in) ...,2020/9,-414823101
160,2330,現金流量表,CCCC,籌資活動之淨現金流入（流出） Net cash flows from (used in) ...,2020/9,12588708
161,2330,現金流量表,DDDD,匯率變動對現金及約當現金之影響Effect of exchange rate changes...,2020/9,-12505458
162,2330,現金流量表,EEEE,本期現金及約當現金增加（減少）數Net increase (decrease) in cas...,2020/9,148795777
